In [209]:
from bs4 import BeautifulSoup
from ics import Calendar, Event
import urllib.request
import datetime
""" 
Länd autmoatisch alle Kalenderinträge runter und speichert sie als ical file
hab zuerst versucht ort und Zeit richitg zu filtern und zu speichern gab aber probleme mit
den Zeitzonen von google calendar jetzt ganzätiges event mit beschreibung
beschreibung passt nicht bei allen z.b. blutspende
es wird viel gefiltert was nicht mehr so nötig ist
Verbesserung: alles unter dem Titel der veranstatlung als beschreibung speichern
"""
MONATE = ("Jaenner","Februar","Maerz","April","Mai","Juni","Juli",
          "August","September","Oktober","November","Dezember")
JAHR = 2020
URL = "http://www.neusiedlamsee.at/gemeinde/buergerservice/termine-veranstaltungen/"
FILE = "Neusiedl.ics"

#soup = BeautifulSoup(html_doc, 'html.parser')

# for monat in MONATE:
#     url_complete = URL+monat+"-"+str(JAHR)
#     print(url_complete)
kalender = Calendar()

for monat_nr, monat in enumerate(MONATE):
  
    url_complete = URL+monat+"-"+str(JAHR)
    print(url_complete)
    try:
        with urllib.request.urlopen(url_complete) as f:
            html_doc = f.read()
    except:
        pass
    if monat == "Jaenner":
        continue

    soup = BeautifulSoup(html_doc, 'html.parser')
    r = soup.select('#rechts #inhalt')
    r = r[0]
    ts = r.find_all('table')

    for t in ts:
        for trs in t.find_all('tr'):
            print("----")
            for i,td in enumerate(trs.find_all('td')):
                if i == 0:
                    tag = str(td.span.contents[0])
                    #print("Tag::", tag)
                if i == 1:
                    tag_nr_von, tag_nr_bis = ("","")
                    nr = str(td.span.contents[0])
                    if '-' in nr or '&' in nr:
                        if '-' in nr:
                            tag_nr_von, tag_nr_bis = nr.split('-')
                        if '&' in nr:
                            tag_nr_von, tag_nr_bis = nr.split('&') 
                        tag_nr_von = tag_nr_von.split('.')[0]
                        tag_nr_bis = tag_nr_bis.split('.')[0]
                        print(tag_nr_von,"--", tag_nr_bis)
                    else:
                        tag_nr_von = nr.split('.')[0]
                        #print("#::", tag_nr_von)
                    tag_nr_von = tag_nr_von.strip()
                    tag_nr_bis = tag_nr_bis.strip()
                if i == 2:
                    was = str(td.h2.string)
                    zeit = ""
                    wo = ""
                    is_1 = ""
                    is_2 = ""
                    try: 
                        is_1 = td.p.span.next
                        is_1 = is_1.strip()
                        if is_1 == "Wo:":
                            wo = str(td.p.span.next.next)
                            print("Wo1", wo)
                        if is_1 == "Wann:":
                            zeit = str(td.p.span.next.next) 
                            zeit = zeit.strip().split(' ')[0]
                            print("zeit1", zeit)
                    except:
                        pass
                    try:
                        is_2 = td.p.next_sibling.span.next
                        is_2 = is_2.strip()
                        if is_2 == "Wo:":
                            wo = str(td.p.next_sibling.span.next.next) 
                            print("wo2", wo)
                        if is_2 == "Wann:":
                            zeit = str(td.p.next_sibling.span.next.next)  
                            print("zeit2", zeit)
                    except:
                        pass
            stunde = ""
            minute = ""
            beschreibung = ""

            if is_1 == "Wo:" or is_2 == "Wo:":
                beschreibung = "Wo: " + wo + "\n"
            if is_1 == "Wann:" or is_2 == "Wann:":
                beschreibung = beschreibung + "Wann: "+ zeit
                
            
            try:
                zeit_von = zeit.split('-')[0]
                stunde,minute = zeit_von.split(':')
                datum = datetime.datetime(int(JAHR), monat_nr+1, int(tag_nr_von),
                                      hour = int(stunde), minute = int(minute),tzinfo = timezone.cet)
                print("stunde/min",stunde,"/",minute)
            except:
                datum = datetime.datetime(int(JAHR),monat_nr+1, int(tag_nr_von))

            #print("JETZT;;", datetime.datetime.now())
            print("Was::", was)
            print("Datum:", datum)
            #print("monat", monat, " ", monat_nr+1 )
            #print("tagnr_von::", tag_nr_von)
            #print("tagnr_bis::", tag_nr_bis)            
            #print("zeit:: ", zeit)        
            #print("wo;; ", wo)
            print("beschreibung: ", beschreibung)
            
            if tag_nr_bis == "" :
                e = Event(name = was, begin = datum, created = datetime.datetime.now(), description = beschreibung)
                e.make_all_day()
                #e = Event(name = was, location = wo, begin = datum, created =  datetime.datetime.now(), description = beschreibung )#duration = {"hours":1} )
            if tag_nr_bis > tag_nr_von:
                datum_bis = datetime.datetime(int(JAHR),monat_nr+1, int(tag_nr_bis))
                e = Event(name = was, begin = datum, end = datum_bis,  created =  datetime.datetime.now(), description = beschreibung )
                e.make_all_day()
                #e = Event(name = was, location = wo, begin = datum, end = datum_bis,  created =  datetime.datetime.now(), description = beschreibung )
            kalender.events.add(e)
display(kalender.events)
with open(FILE, 'w') as my_file:
    my_file.writelines(kalender)

                    #print("XX1",td.p.span.next)
                    #print("XX2",td.p.next_sibling.span.next)
    #                 try:
    #                     wo = str(td.find_all('p')[1].contents[1])                 
    #                 except:
    #                     pass
    #                 try:
    #                     zeit = str(td.find_all('p')[0].contents[1])
    #                     zeit = zeit.strip().split(' ')[0]
    #                 except:
    #                     pass

http://www.neusiedlamsee.at/gemeinde/buergerservice/termine-veranstaltungen/Jaenner-2020
http://www.neusiedlamsee.at/gemeinde/buergerservice/termine-veranstaltungen/Februar-2020
----
zeit1 15:00
wo2 Zur alten Mauth
Was:: Faschingskränzchen der Senioren
Datum: 2020-02-11 00:00:00
beschreibung:  Wo: Zur alten Mauth
Wann: 15:00
----
zeit2  19:00 Uhr
Was::  Faschingsparty
Datum: 2020-02-15 00:00:00
beschreibung:  Wann:  19:00 Uhr
----
zeit1 19:00
wo2 Weinwerk - Vino Saal
Was:: Themenabend der evang. Kirche - Altes neu entdeckt im Neusiedler Stadtarchiv
Datum: 2020-02-17 00:00:00
beschreibung:  Wo: Weinwerk - Vino Saal
Wann: 19:00
----
zeit1 19:00
wo2 Zum echten Leben
Was:: Weinstammtisch Weingut GERNOT HEINRICH – „Natur und Kultur“
Datum: 2020-02-20 00:00:00
beschreibung:  Wo: Zum echten Leben
Wann: 19:00
----
zeit1 14:00
wo2 entlang der Hauptstraße
Was:: Faschingsumzug der katholischen Jugend
Datum: 2020-02-22 00:00:00
beschreibung:  Wo: entlang der Hauptstraße
Wann: 14:00
----
Wo1 Tanzsc

{<all-day Event 'Faschingskränzchen der Senioren' 2020-02-11>,
 <all-day Event ' Faschingsparty' 2020-02-15>,
 <all-day Event 'Themenabend der evang. Kirche - Altes neu entdeckt im Neusiedler Stadtarchiv' 2020-02-17>,
 <all-day Event 'Weinstammtisch Weingut GERNOT HEINRICH – „Natur und Kultur“' 2020-02-20>,
 <all-day Event 'Faschingsumzug der katholischen Jugend' 2020-02-22>,
 <all-day Event 'Kinderfaschingsfest' 2020-02-22>,
 <all-day Event 'Faschingsfest' 2020-02-25>,
 <all-day Event 'Aschermittwoch' 2020-02-26>,
 <all-day Event 'MOKI Familienzirkel' 2020-02-27>,
 <all-day Event 'Monatsmarkt' 2020-03-03>,
 <all-day Event 'Diavortrag Helmut Pichler "Afrika"' begin:2020-03-05 end:2020-03-07>,
 <all-day Event 'Ökumenischer Weltgebetstag der Frauen' 2020-03-06>,
 <all-day Event 'Hallenvolleyballturnier' 2020-03-07>,
 <all-day Event 'Kunstflohmarkt & Ausstellung' begin:2020-03-07 end:2020-03-09>,
 <all-day Event 'Trommelworkshop für Frauen' 2020-03-08>,
 <all-day Event 'Weinstammtisch PFA